In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML,clear_output


In [2]:
!pip install pycaret==2.3.10

     |████████████████████████████████| 320 kB 786 kB/s 
     |████████████████████████████████| 167 kB 11.7 MB/s 
     |████████████████████████████████| 17.9 MB 11.1 MB/s 
     |████████████████████████████████| 24.2 MB 46.7 MB/s 
     |████████████████████████████████| 25.9 MB 15.1 MB/s 
     |████████████████████████████████| 125 kB 62.9 MB/s 
     |████████████████████████████████| 81 kB 5.6 MB/s 
     |████████████████████████████████| 79 kB 6.3 MB/s 
  Created wheel for databricks-cli: filename=databricks_cli-0.17.0-py3-none-any.whl size=141959 sha256=4553a8ae5c34928e52a009b218af034fc2d92c62991e2b661364f39c7e37ba1b
  Stored in directory: /root/.cache/pip/wheels/55/c3/db/33705569425fd2bdc9ea73051a8053fa26965c2bce8a146747
  Created wheel for pyod: filename=pyod-1.0.3-py3-none-any.whl size=154724 sha256=ef2455b9811533c49d0eba1d16e5b50eb0cbd3013f5b9efc89d0297384e1d99c
  Stored in directory: /root/.cache/pip/wheels/60/1e/04/b42e786399bd7503af674d1ec95d93665b1700309ec3525b65
Successfu

In [3]:
from pycaret.regression import *

In [4]:
productUserdf = pd.read_csv('../input/featureextraction-nowindow-digikala/productUsertrdf.csv')
targetdf = pd.read_csv('../input/featureextraction-nowindow-digikala/targetLastDatePurchase_user-product.csv')
print(productUserdf.shape)
print(targetdf.shape)

(318050, 16)
(318050, 7)


In [5]:
productUserdf.head(2)

,Unnamed: 0,user_id,product_id,allDiffDay,LenOfList,allGoneDay,ProductProbPerUser,ProductProb,avgDiffDay,stdDiffDay,maxDiffDay,minDiffDay,maxGoneDay,minGoneDay,medianGoneDay,stdGoneDay
0,0,577243,724112,"[nan, 12.0, 28.0, 55.0]",4,"[71, 83, 111, 166]",0.028571,0.0027,31.666667,17.745109,55.0,12.0,166,71,97.0,36.629053
1,1,577243,875550,"[nan, 25.0, 59.0]",3,"[18, 43, 102]",0.021429,0.0694,42.000000,17.000000,59.0,25.0,102,18,43.0,35.216789


In [6]:
targetdf.head(2)

,Unnamed: 0,index,user_id,product_id,created_at,diffDay,GoneDays
0,0,988009,577243,724112,2019-06-29,13.0,179
1,1,1521046,577243,875550,2019-06-16,64.0,166


In [7]:
targetdf.rename(columns = {'created_at':'created_at_target', 'diffDay':'diffDay_target',
                           'GoneDays':'GoneDays_target',
                          'index':'index_target'}, inplace = True)
targetdf.head(2)

,Unnamed: 0,index_target,user_id,product_id,created_at_target,diffDay_target,GoneDays_target
0,0,988009,577243,724112,2019-06-29,13.0,179
1,1,1521046,577243,875550,2019-06-16,64.0,166


> GoneDay is calculated based on 2019-01-01

In [8]:
df = pd.merge(left=productUserdf, right=targetdf,how='left',on=['user_id','product_id'])
df.drop(['Unnamed: 0_x','Unnamed: 0_y'],inplace=True,axis=1)
df.head(2)

,user_id,product_id,allDiffDay,LenOfList,allGoneDay,ProductProbPerUser,ProductProb,avgDiffDay,stdDiffDay,maxDiffDay,minDiffDay,maxGoneDay,minGoneDay,medianGoneDay,stdGoneDay,index_target,created_at_target,diffDay_target,GoneDays_target
0,577243,724112,"[nan, 12.0, 28.0, 55.0]",4,"[71, 83, 111, 166]",0.028571,0.0027,31.666667,17.745109,55.0,12.0,166,71,97.0,36.629053,988009,2019-06-29,13.0,179
1,577243,875550,"[nan, 25.0, 59.0]",3,"[18, 43, 102]",0.021429,0.0694,42.000000,17.000000,59.0,25.0,102,18,43.0,35.216789,1521046,2019-06-16,64.0,166


In [9]:
data = df.sample(frac=0.95,random_state=2)
data_unseen = df.drop(data.index)
print(data.shape)
print(data_unseen.shape)

(302148, 19)
(15902, 19)


In [10]:
ignore_features =['user_id','product_id','allDiffDay','allGoneDay','index_target','created_at_target','diffDay_target']
# s = setup(data=data,target='GoneDays_target',ignore_features =ignore_features,normalize=True,normalize_method='minmax',use_gpu=True)
s = setup(data=data,target='GoneDays_target',ignore_features =ignore_features,use_gpu=True,silent=True)

,Description,Value
0,session_id,4660
1,Target,GoneDays_target
2,Original Data,"(302148, 19)"
3,Missing Values,False
4,Numeric Features,11
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(211503, 11)"


In [11]:
models()

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,cuml.linear_model.linear_regression.LinearRegr...,True
lasso,Lasso Regression,cuml.linear_model.lasso.Lasso,True
ridge,Ridge Regression,cuml.linear_model.ridge.Ridge,True
en,Elastic Net,cuml.linear_model.elastic_net.ElasticNet,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


In [12]:
model = compare_models(include = ['rf','xgboost', 'lightgbm','catboost'],sort='MSE',n_select=4)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,23.0464,1110.7600,33.3274,0.8168,0.1881,0.1227,1.8410
catboost,CatBoost Regressor,23.1710,1120.1774,33.4683,0.8152,0.1893,0.1232,12.3240
xgboost,Extreme Gradient Boosting,23.0831,1125.3175,33.5451,0.8144,0.1907,0.1231,1.4690
rf,Random Forest Regressor,23.3785,1140.0837,33.7644,0.8120,0.1928,0.1260,3.2990


In [13]:
model

[LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               device='gpu', importance_type='split', learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
               objective=None, random_state=4660, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0),
 XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
              objective='reg:squarederror', random_state=4660, reg_alpha=0,
              reg

In [14]:
#we can use gpu without any additional configurations.
catboostmodel = create_model('catboost',return_train_score=True)

In [15]:
#we need some additional configurations to using gpu in this model, then i prefer to use carboost model.
#ctmodel = create_model('lightgbm',return_train_score=True)

In [16]:
tunedModel = tune_model( catboostmodel,return_train_score=True,optimize='MSE',n_iter=50,choose_better=False)

In [17]:
predict_model(tunedModel,data=data_unseen)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,23.0576,1110.2541,33.3205,0.8173,0.1899,0.1232


,user_id,product_id,allDiffDay,LenOfList,allGoneDay,ProductProbPerUser,ProductProb,avgDiffDay,stdDiffDay,maxDiffDay,minDiffDay,maxGoneDay,minGoneDay,medianGoneDay,stdGoneDay,index_target,created_at_target,diffDay_target,GoneDays_target,Label
85,578067,1764220,"[nan, 10.0, 9.0, 14.0, 10.0]",5,"[185, 195, 204, 218, 228]",1.000000,0.0019,10.750000,1.920286,14.0,9.0,228,185,204.0,15.453155,1325214,2019-08-24,7.0,235,257.242422
92,578185,629577,"[nan, 60.0, 23.0]",3,"[235, 295, 318]",1.000000,0.0074,41.500000,18.500000,60.0,23.0,318,235,295.0,34.988887,805511,2019-12-06,21.0,339,336.706177
95,578330,840895,"[nan, 37.0, 84.0, 101.0]",4,"[17, 54, 138, 239]",0.235294,0.0014,74.000000,27.067816,101.0,37.0,239,17,96.0,85.431259,1431104,2019-08-28,0.0,239,284.088588
106,578544,2593993,"[nan, 9.0, 59.0, 2.0, 118.0]",5,"[37, 46, 105, 107, 225]",0.200000,0.0006,47.000000,46.513439,118.0,2.0,225,37,105.0,67.088002,377417,2019-09-05,22.0,247,263.553028
113,578562,1995625,"[nan, 112.0, 42.0]",3,"[56, 168, 210]",0.125000,0.0093,77.000000,35.000000,112.0,42.0,210,56,168.0,64.999145,794472,2019-09-07,39.0,249,257.636600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317953,44771212,4643651,"[nan, 9.0, 10.0, 11.0, 9.0]",5,"[122, 131, 141, 152, 161]",0.034247,0.0027,9.750000,0.829156,11.0,9.0,161,122,141.0,14.008569,913715,2019-06-20,9.0,170,188.389736
317993,44999886,2272494,"[nan, 22.0, 14.0, 8.0, 67.0, 141.0, 11.0, 7.0,...",9,"[19, 41, 55, 63, 130, 271, 282, 289, 335]",0.051429,0.1060,39.500000,43.182751,141.0,7.0,335,19,130.0,120.010185,1717971,2019-12-24,22.0,357,341.961180
318016,45325487,778755,"[nan, 0.0, 0.0, 33.0, 0.0, 0.0, 50.0, 0.0, 20....",10,"[111, 111, 111, 144, 144, 144, 194, 194, 214, ...",0.294118,0.0408,11.444444,17.670510,50.0,0.0,214,111,144.0,40.098504,1183004,2019-08-03,0.0,214,238.247455
318028,45334805,535996,"[nan, 0.0, 0.0, 73.0, 0.0, 22.0, 0.0, 10.0]",8,"[194, 194, 194, 267, 267, 289, 289, 299]",0.195122,0.0278,15.000000,24.894061,73.0,0.0,299,194,267.0,43.907395,2168312,2019-10-27,0.0,299,311.676030
